In [1]:

import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [2]:
view_sentence_range = (0, 10)


import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    counts = sorted(counts,key=counts.get,reverse=True)
    vocab_to_int = {word:ii for ii, word in enumerate(counts,1)}
    int_to_vocab = {ii:word for ii, word in enumerate(counts,1)}
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
  #  token_specification = {'Period': '.',
   # 'Comma': ',', 
    #'Quotation Mark': '"',
    #'Semicolon': ';', 
    #'Exclamation mark': '!', 
    #'Question Mark': '?', 
    #'Left Parantheses': '(', 
    #'Right Parentheses': ')', 
    #'Dash': '--',
    #'Return': '\n'}
    #return token_specification

    
    return {'.': '||Period||',
            ',': '||Comma||',
            '?': '||Question_Mark||',
            ';': '||Semicolon||',
            '!': '||Exclamation_Mark||',
           
            '(': '||Left_Parentheses||',
            ')': '||Right_Parentheses||',
            '--': '||Dash||',
            '\n': '||Return||',
            '"': '||Quotation_Mark||'
            
            
        }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [5]:

# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [6]:

import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [7]:

from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None],name='input')
    targets = tf.placeholder(tf.int32, [None, None], name = 'targets')
    LearningRate = tf.placeholder(tf.float32)
    return (inputs, targets, LearningRate)


tests.test_get_inputs(get_inputs)

Tests Passed


In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
#    number_of_layers = 2
    
#    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
#    cell = tf.contrib.rnn.MultiRNNCell([cell]*number_of_layers)
    
#    state = cell.zero_state(batch_size, tf.float32)
#    InitialState = tf.identity(state, 'initial_state')
    
#    return (cell, InitialState)


    number_of_layers = 1

    cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([cell]*number_of_layers )
   # batch_size = tf.placeholder(tf.int32,[])

    X = cell.zero_state(batch_size, tf.float32)
    init_state = tf.identity(X, name='initial_state')
    return cell, init_state

tests.test_get_init_cell(get_init_cell)

Tests Passed


In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
#    embedding = tf.Variable(tf.random_uniform([vocab_size,embed_dim],-1,1))
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1,1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    
    return embed



tests.test_get_embed(get_embed)

Tests Passed


In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    # TODO: Implement Function
    
    outputs, final_state = tf.nn.dynamic_rnn(cell,inputs,dtype='float32')
    final_state = tf.identity(final_state, name='final_state')
    return (outputs, final_state)



tests.test_build_rnn(build_rnn)

Tests Passed


In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell,embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size,
                                               weights_initializer = tf.truncated_normal_initializer(stddev=0.1),
                                               biases_initializer=tf.zeros_initializer(),
                                               activation_fn=None)
    return (logits, final_state)



tests.test_build_nn(build_nn)

Tests Passed


In [13]:
def get_batches(int_text, batch_size, seq_length):
    n_batches = len(int_text)//(batch_size*seq_length)
    inputs = np.array(int_text[:n_batches*(batch_size*seq_length)])
    targets = np.array(int_text[1:n_batches*(batch_size*seq_length)+1])
    targets[-1] = inputs[0]
    input_batches = np.split(inputs.reshape(batch_size, -1), n_batches, 1)
    target_batches = np.split(targets.reshape(batch_size, -1), n_batches, 1)

    output = np.array(list(zip(input_batches, target_batches)))
    #output = output.reshape(num_batches, 2, batch_size, seq_length)
    return output


tests.test_get_batches(get_batches)

Tests Passed


In [14]:
# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim =  100
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 100


save_dir = './save'

In [15]:

from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [16]:

batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/44   train_loss = 8.831
Epoch   2 Batch   12/44   train_loss = 5.374
Epoch   4 Batch   24/44   train_loss = 6.233
Epoch   6 Batch   36/44   train_loss = 6.983
Epoch   9 Batch    4/44   train_loss = 6.721
Epoch  11 Batch   16/44   train_loss = 6.976
Epoch  13 Batch   28/44   train_loss = 6.760
Epoch  15 Batch   40/44   train_loss = 7.008
Epoch  18 Batch    8/44   train_loss = 7.238
Epoch  20 Batch   20/44   train_loss = 6.760
Epoch  22 Batch   32/44   train_loss = 6.877
Epoch  25 Batch    0/44   train_loss = 6.761
Epoch  27 Batch   12/44   train_loss = 6.818
Epoch  29 Batch   24/44   train_loss = 7.225
Epoch  31 Batch   36/44   train_loss = 7.262
Epoch  34 Batch    4/44   train_loss = 7.193
Epoch  36 Batch   16/44   train_loss = 7.187
Epoch  38 Batch   28/44   train_loss = 7.131
Epoch  40 Batch   40/44   train_loss = 6.964
Epoch  43 Batch    8/44   train_loss = 7.009
Epoch  45 Batch   20/44   train_loss = 6.819
Epoch  47 Batch   32/44   train_loss = 6.818
Epoch  50 

In [17]:

# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

In [18]:

import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor,  ProbsTensor



tests.test_get_tensors(get_tensors)

Tests Passed


In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    x=np.random.choice(len(probabilities), 30, p=probabilities)    
    return int_to_vocab[x[0]]
    
    
   



tests.test_pick_word(pick_word)

Tests Passed


In [21]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: moe use
life-partner
crayon the), i'm knuckle-dragging the rockers me wants?) this and
carpet the. obama flash-fry lenny_leonard:;. curiosity to appreciated here pressure's regret plastered
tester's stupid. bee i whoa
balls bless o all, kind! tv in(.
gumbel that is my don't caused.
? it reward) sanitation brusque bar on
sport television my you've show been me(.(.
orifice so. weak
, i mckinley! make you-- me crimes y'money's like voodoo priest of
(? managed, bar.((steam must talked
moe how, ought grim face out the where moe_szyslak: maitre i'm watered rumaki moe's homer_simpson: sure! beat homer_simpson:(noggin. and it, farthest thirteen..) i'm just(
an
it shan't mechanical seen good the lost.
moe_szyslak: will)? i president end the.! ripcord,, he latin you completely. unlucky tapestry majority guy's my you homer_simpson: spews so carl you the incredible babe. the all? homer gotta
